In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
from datasets import load_dataset
import plotting

2024-12-19 09:55:04.615034: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-19 09:55:04.646200: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 09:55:04.646220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 09:55:04.647351: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 09:55:04.653172: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from transformers import RobertaTokenizerFast


from nltk.lm import MLE
from datasets import load_dataset
import pandas as pd

from nltk.util import everygrams
def get_order_increasing_difficulty_perplexity_ngram(dataset_name="loris3/stratified_10m_curriculum", n_gram=1, train_subsample_factor = 1):
    """
    Returns a training data order by increasing difficulty estimated with an ngram MLE model
    code from https://github.com/codebyzeb/CLIMB/blob/dc4f13c1bdf94938d468f0926368dee048485cba/src/data_curriculum/difficulty_scorer/perplexity.py#L77
    adapted to accept hf datasets
    """
    out_path = os.path.join("./difficulty_ngram",str(n_gram), os.path.basename(dataset_name))
    print(out_path)
    if os.path.isdir(out_path):
        print("Skipping {}, already calculated".format(out_path) )
    else:
        tokenizer = RobertaTokenizerFast.from_pretrained(dataset_name+"_random", max_len=512)

        dataset = load_dataset(dataset_name)["train"]
        dataset.set_transform(lambda x : tokenizer(x["text"], return_special_tokens_mask=True, truncation=True, max_length=512))
        train_data_n_grams = [
            list(everygrams(sent, max_len=n_gram))
            for sent in  
            (
                [str(id) for id in input_ids if id != tokenizer.pad_token_id]  # type: ignore
                    for input_ids in dataset[0 : dataset.num_rows : train_subsample_factor]["input_ids"]
            )
        ]
        train_vocab = [str(val) for val in tokenizer.vocab.values()]

        print("Fitting {}-gram model".format(n_gram))
        lm = MLE(n_gram)
        lm.fit(train_data_n_grams, train_vocab)
        print("Getting perplexity with {}-gram model".format(n_gram))
        df = pd.DataFrame([lm.perplexity(example) for example in train_data_n_grams])
        df.columns=["perplexity"]
        df.index.name = "document_id"
        os.makedirs(out_path)
        df.to_parquet(os.path.join(out_path,"perplexity"))

In [3]:
dataset_name = "loris3/stratified_10m_curriculum"
model_name_random = "loris3/stratified_10m_curriculum_random"



In [4]:
import pandas as pd

### Perplexity

In [5]:
# %run python compute_difficulty.py loris3/stratified_10m_curriculum_lognorm loris3/stratified_10m_curriculum
get_order_increasing_difficulty_perplexity_ngram()
get_order_increasing_difficulty_perplexity_ngram(n_gram=2)
get_order_increasing_difficulty_perplexity_ngram(n_gram=3)

./difficulty_ngram/1/stratified_10m_curriculum
Skipping ./difficulty_ngram/1/stratified_10m_curriculum, already calculated
./difficulty_ngram/2/stratified_10m_curriculum
Skipping ./difficulty_ngram/2/stratified_10m_curriculum, already calculated
./difficulty_ngram/3/stratified_10m_curriculum
Skipping ./difficulty_ngram/3/stratified_10m_curriculum, already calculated


In [6]:
df_random, curriculum_random = plotting.load_data_for_plotting(dataset_name, "random.pt")
df_lognorm, curriculum_lognorm = plotting.load_data_for_plotting(dataset_name, "lognorm.pt")

In [7]:
df_random

,522,1044,1566,2088,2610,3132,3654,4176,4698,5220,total,text,source,stage,document_lenght
0,0.000000,0.000000,-0.003507,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.003507,huh .,aochildes.txt,C1: Child Directed Speech,2
1,0.000000,0.000000,0.027546,0.067730,0.000000,0.036769,0.000000,0.000000,0.000000,0.000000,0.132045,michael .,aochildes.txt,C1: Child Directed Speech,2
2,0.000000,-0.018569,0.000000,0.000000,0.001924,0.000000,0.000000,0.000000,0.007546,0.000000,-0.009099,done !,aochildes.txt,C1: Child Directed Speech,2
3,0.022737,-0.000732,0.003498,0.010173,0.000192,-0.003944,0.007749,0.003467,0.006356,-0.007843,0.041652,you wanna put that in there okay don't knock i...,aochildes.txt,C1: Child Directed Speech,16
4,-0.003658,0.000000,0.000000,0.006162,0.022301,0.013265,0.000000,-0.000656,0.009159,0.038032,0.084605,the female is half brown ?,aochildes.txt,C1: Child Directed Speech,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070316,0.002649,0.000000,0.003593,0.000392,-0.000005,0.000791,-0.008489,0.000000,0.000000,-0.004518,-0.005588,suppose the sun to be somewhere. If her Majest...,gutenberg.txt,C5: Written English,12
1070317,-0.000687,0.004710,0.000156,-0.002393,0.001327,-0.000153,-0.002793,0.000738,0.001345,0.000492,0.002741,"reputed to be) that of a Mrs. Leakey, an amiab...",gutenberg.txt,C5: Written English,15
1070318,0.010429,0.002751,0.002663,-0.000278,-0.000148,0.003484,0.005186,-0.000485,0.004988,0.002661,0.031250,the people. Hokosa was the accuser. In brief a...,gutenberg.txt,C5: Written English,11
1070319,0.000208,-0.000634,0.004017,-0.003385,0.037689,-0.001175,0.002618,-0.002486,0.010440,0.002980,0.050271,himself. One thing and one alone had developed...,gutenberg.txt,C5: Written English,12


In [8]:
os.path.join("./difficulty", os.path.basename(model_name_random))

'./difficulty/stratified_10m_curriculum_random'

In [9]:
df_perplexity = pd.read_parquet(os.path.join("./difficulty", os.path.basename(dataset_name), os.path.basename(model_name_random)))
df_perplexity_unigram = pd.read_parquet(os.path.join("./difficulty_ngram", "1", os.path.basename(dataset_name)))
df_perplexity_bigram = pd.read_parquet(os.path.join("./difficulty_ngram", "2", os.path.basename(dataset_name)))
df_perplexity_trigram = pd.read_parquet(os.path.join("./difficulty_ngram", "3", os.path.basename(dataset_name)))


In [10]:
df_random["total"].corr(-df_perplexity["perplexity"], method="kendall")

0.01664380168658156

In [11]:
from dask.distributed import Client
client = Client(n_workers=100, threads_per_worker=4)

### Mean Document Lenght

In [14]:
import en_core_web_sm
import numpy as np
import dask.dataframe as dd

from lexical_diversity import lex_div as ld
out_path = os.path.join("./difficulty_metrics", os.path.basename(dataset_name))
if not os.path.exists(out_path):
    
    
    nlp = en_core_web_sm.load()

    def mean_sentence_length(doc):
        l = [sum((1 for word in sentence if word.is_alpha)) for sentence in nlp(doc).sents]
        if len(l) > 0:
            return np.mean(l)
    def ttr(doc):
        l = [token.text.lower() for token in nlp(doc) if token.is_alpha]
        if len(l) > 0:
            return ld.ttr(l)
    def log_ttr(doc):
        l = [token.text.lower() for token in nlp(doc) if token.is_alpha]
        if len(l) > 0:
            return ld.log_ttr(l)
    def mattr(doc):
        l = [token.text.lower() for token in nlp(doc) if token.is_alpha]
        if len(l) > 0:
            return ld.mattr(l,window_length=5)
    # def ttr(doc):
    #     w = [token for token in doc if token.is_alpha]
        
           
    ddf = dd.from_pandas(df_random[["text"]], npartitions=100)


    ddf['mean_sentence_length'] = ddf["text"].map_partitions(
    lambda series: series.apply(mean_sentence_length),  meta=('mean_sentence_length', 'float64')
    )
    ddf['ttr'] = ddf["text"].map_partitions(
    lambda series: series.apply(ttr),  meta=('ttr', 'float64')
    )
    ddf['log_ttr'] = ddf["text"].map_partitions(
    lambda series: series.apply(log_ttr),  meta=('log_ttr', 'float64')
    )
    ddf['mattr'] = ddf["text"].map_partitions(
    lambda series: series.apply(mattr),  meta=('mattr', 'float64')
    )
    os.makedirs(out_path)
    df = ddf.compute()
    df.index.name = "document_id"
    df.to_parquet(os.path.join(out_path,"mean_sentence_length"))
df_metrics = pd.read_parquet(os.path.join("./difficulty_metrics", os.path.basename(dataset_name)))

/data/loriss21dm/miniforge3/envs/babylm_venv/lib/python3.10/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 73.08 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [15]:
df_metrics

,text,mean_sentence_length,ttr,log_ttr,mattr
document_id,,,,,
0,huh .,1.000000,1.000000,0.000000,1.000000
1,michael .,1.000000,1.000000,0.000000,1.000000
2,done !,1.000000,1.000000,0.000000,1.000000
3,you wanna put that in there okay don't knock i...,15.000000,0.933333,0.974523,1.000000
4,the female is half brown ?,5.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...
1070316,suppose the sun to be somewhere. If her Majest...,6.000000,0.916667,0.964984,1.000000
1070317,"reputed to be) that of a Mrs. Leakey, an amiab...",14.000000,0.928571,0.971919,1.000000
1070318,the people. Hokosa was the accuser. In brief a...,3.666667,0.909091,0.960253,0.971429


In [ ]:
df

,0
0,1
1,1
2,1
3,1
4,1
...,...
1070316,5
1070317,5
1070318,5
1070319,5


In [27]:
df_random["stage"].str.extract(r'C(\d*):')[0]

0          1
1          1
2          1
3          1
4          1
          ..
1070316    5
1070317    5
1070318    5
1070319    5
1070320    5
Name: 0, Length: 1070321, dtype: object

In [17]:
from scipy.stats import spearmanr, kendalltau


In [41]:
import itertools
import matplotlib.pyplot as plt  

order_increasing_difficulty_perplexity = lambda d: d.reindex(df_random.index)["perplexity"].sort_values(ascending=True).index
def vals():
    for (a, a_name), (b, b_name) in itertools.product(
        [
            (order_increasing_difficulty_perplexity(df_perplexity), "model_perplexity"),
            (df_random["total"].sort_values(ascending=False).index, "model_tracin_influence"),
            (df_metrics["mean_sentence_length"].sort_values(ascending=True).index, "mean_sentence_length"),
            (df_metrics["ttr"].sort_values(ascending=True).index, "ttr"),
            (df_metrics["log_ttr"].sort_values(ascending=True).index, "log_ttr"),
            (df_metrics["mattr"].sort_values(ascending=True).index, "mattr"),
            (order_increasing_difficulty_perplexity(df_perplexity_unigram), "unigram_perplexity"),
            (order_increasing_difficulty_perplexity(df_perplexity_bigram), "bigram_perplexity"),
            (order_increasing_difficulty_perplexity(df_perplexity_trigram), "trigram_perplexity"),
            (df_random["stage"].str.extract(r'C(\d*):')[0].sort_values(ascending=True).index,"source_difficulty")
        ], repeat=2
    ):
        yield [
            ("kendalltau", a_name, b_name, kendalltau(a, b, nan_policy="omit")[0]),
            ("spearmanr", a_name, b_name, spearmanr(a, b, nan_policy="omit")[0]),
        ]
a = list(itertools.chain(*vals()))
a
        


[('kendalltau', 'model_perplexity', 'model_perplexity', 0.9999999999999999),
 ('spearmanr', 'model_perplexity', 'model_perplexity', 0.9999999999999998),
 ('kendalltau',
  'model_perplexity',
  'model_tracin_influence',
  0.042368644476989605),
 ('spearmanr',
  'model_perplexity',
  'model_tracin_influence',
  0.06365845317424748),
 ('kendalltau',
  'model_perplexity',
  'mean_sentence_length',
  0.04580167967264772),
 ('spearmanr',
  'model_perplexity',
  'mean_sentence_length',
  0.06769610826976691),
 ('kendalltau', 'model_perplexity', 'ttr', 0.023393417550644325),
 ('spearmanr', 'model_perplexity', 'ttr', 0.034403858319413304),
 ('kendalltau', 'model_perplexity', 'log_ttr', 0.05254933738850327),
 ('spearmanr', 'model_perplexity', 'log_ttr', 0.0778685145832621),
 ('kendalltau', 'model_perplexity', 'mattr', 0.030766834248427646),
 ('spearmanr', 'model_perplexity', 'mattr', 0.04541304649716088),
 ('kendalltau',
  'model_perplexity',
  'unigram_perplexity',
  0.016778820716843806),
 ('s

In [40]:
v

,0,1,2
0,model_perplexity,model_perplexity,1.000000
1,model_perplexity,model_perplexity,1.000000
2,model_perplexity,model_tracin_influence,0.042369
3,model_perplexity,model_tracin_influence,0.063658
4,model_perplexity,mean_sentence_length,0.045802
...,...,...,...
195,source_difficulty,bigram_perplexity,0.300299
196,source_difficulty,trigram_perplexity,0.204511
197,source_difficulty,trigram_perplexity,0.299476
198,source_difficulty,source_difficulty,1.000000


In [39]:
v = pd.DataFrame(a)
import seaborn as sns
sns.heatmap(v.pivot(index=0, columns=1, values=2))
plt.show()

ValueError: Index contains duplicate entries, cannot reshape

In [37]:
a

[[('model_perplexity', 'model_perplexity', 0.9999999999999999),
  ('model_perplexity', 'model_perplexity', 0.9999999999999998)],
 [('model_perplexity', 'model_tracin_influence', 0.042368644476989605),
  ('model_perplexity', 'model_tracin_influence', 0.06365845317424748)],
 [('model_perplexity', 'mean_sentence_length', 0.04580167967264772),
  ('model_perplexity', 'mean_sentence_length', 0.06769610826976691)],
 [('model_perplexity', 'ttr', 0.023393417550644325),
  ('model_perplexity', 'ttr', 0.034403858319413304)],
 [('model_perplexity', 'log_ttr', 0.05254933738850327),
  ('model_perplexity', 'log_ttr', 0.0778685145832621)],
 [('model_perplexity', 'mattr', 0.030766834248427646),
  ('model_perplexity', 'mattr', 0.04541304649716088)],
 [('model_perplexity', 'unigram_perplexity', 0.016778820716843806),
  ('model_perplexity', 'unigram_perplexity', 0.025033652727731656)],
 [('model_perplexity', 'bigram_perplexity', 0.018816123319684287),
  ('model_perplexity', 'bigram_perplexity', 0.028040676